In [ ]:
# Instalar librerías necesarias
!pip install transformers torch onnx onnxruntime

# Importar librerías
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import onnx
import onnxruntime as ort
import os
import time

# Cargar el modelo y el tokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

# Texto de prueba
text = "I love using Hugging Face models!"
inputs = tokenizer(text, return_tensors="pt")

# Exportar a ONNX
onnx_model_path = "model.onnx"
torch.onnx.export(
    model,
    (inputs['input_ids'], inputs['attention_mask']),
    onnx_model_path,
    input_names=['input_ids', 'attention_mask'],
    output_names=['logits'],
    opset_version=11
)

print(f"Modelo exportado a ONNX en {onnx_model_path}")

# Cuantizar el modelo ONNX (sin 'quant_format')
from onnxruntime.quantization import quantize_dynamic, QuantType

quantized_model_path = "model_quantized.onnx"
quantize_dynamic(onnx_model_path, quantized_model_path, op_types_to_quantize=['MatMul', 'Add'])
print(f"Modelo cuantizado guardado en {quantized_model_path}")

# Comparar tamaños de archivos en disco
original_size = os.path.getsize(onnx_model_path) / (1024 * 1024)  # Tamaño en MB
quantized_size = os.path.getsize(quantized_model_path) / (1024 * 1024)  # Tamaño en MB

print(f"\nTamaño del modelo ONNX original: {original_size:.2f} MB")
print(f"Tamaño del modelo ONNX cuantizado: {quantized_size:.2f} MB")

# Preparar los datos de entrada para ONNX
input_ids = inputs['input_ids'].numpy()
attention_mask = inputs['attention_mask'].numpy()

# Crear la sesión de inferencia para el modelo original en ONNX
ort_session_original = ort.InferenceSession(onnx_model_path)

# Crear la sesión de inferencia para el modelo cuantizado en ONNX
ort_session_quantized = ort.InferenceSession(quantized_model_path)

# Función para medir el tiempo de inferencia
def measure_inference_time(session, input_ids, attention_mask, num_runs=100):
    start = time.time()
    for _ in range(num_runs):
        session.run(None, {'input_ids': input_ids, 'attention_mask': attention_mask})
    end = time.time()
    avg_time_per_run = (end - start) / num_runs
    return avg_time_per_run

# Medir tiempo de inferencia para el modelo original
time_original = measure_inference_time(ort_session_original, input_ids, attention_mask)
print(f"\nTiempo promedio de inferencia (modelo original): {time_original:.4f} segundos")

# Medir tiempo de inferencia para el modelo cuantizado
time_quantized = measure_inference_time(ort_session_quantized, input_ids, attention_mask)
print(f"Tiempo promedio de inferencia (modelo cuantizado): {time_quantized:.4f} segundos")

# Comparar resultados de inferencia
outputs_quantized = ort_session_quantized.run(None, {'input_ids': input_ids, 'attention_mask': attention_mask})
print("\nSalida del modelo cuantizado:", outputs_quantized[0])





Modelo exportado a ONNX en model.onnx


Modelo cuantizado guardado en model_quantized.onnx

Tamaño del modelo ONNX original: 255.51 MB
Tamaño del modelo ONNX cuantizado: 132.40 MB

Tiempo promedio de inferencia (modelo original): 0.0263 segundos
Tiempo promedio de inferencia (modelo cuantizado): 0.0119 segundos

Salida del modelo cuantizado: [[-3.4982142  3.7058241]]
